<br><br>

# DQLab Internal Data Challenge 3

### Adriyan

<br><br>

In [224]:
import mysql.connector
import pandas as pd
import numpy as np
import math

from bokeh.io import show, output_notebook, output_file
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.palettes import Spectral5, Spectral6
from bokeh.transform import factor_cmap
from bokeh.core.properties import value

output_notebook()

Loading BokehJS ...

[**Relational database of UW_std**](https://relational.fit.cvut.cz/dataset/UW-CSE)

![Relational database of UW_std](https://relational.fit.cvut.cz/assets/img/datasets-generated/UW_std.svg)

<br><br>

## 1. Acquiring database `UW_std` from `host="relational.fit.cvut.cz"`

Creating MySQL connection as `my_conn` 

In [226]:
my_conn = mysql.connector.connect(host="relational.fit.cvut.cz",
                                  port=3306,
                                  user="guest",
                                  passwd="relational",
                                  database="UW_std",
                                  use_pure=True)

Dumping all tables in the database `UW_std` into single csv file:

* Creating a SQL query `my_query` for querying the database `UW_std` 
* Read the database which stored into `pandas.DataFrame` as `my_df` from the query `my_query` and the connection `my_conn`
* Store the acquired database `my_df` into a CSV file: `uw_cse.csv` in `./data` directory 

In [227]:
# Creating a SQL query my_query for querying all tables in the database UW_std
my_query = """
            SELECT p.*, a.p_id_dummy, t.course_id, c.courseLevel
            FROM person AS p
            LEFT OUTER JOIN advisedBy AS a
            ON p.p_id = a.p_id
            LEFT OUTER JOIN taughtBy AS t
            ON p.p_id = t.p_id
            LEFT OUTER JOIN course AS c
            ON t.course_id = c.course_id
           """

# Read the database which stored into pandas.DataFrame as my_df from the query my_query and the connection my_conn
my_df = pd.read_sql_query(my_query, my_conn)

# Store the acquired database my_df into a CSV file: uw_cse.csv, in ./data directory
my_df.to_csv("./data/uw_cse.csv", index=False)


Close the connection `my_conn` and delete `my_df`

In [228]:
my_conn.close()
del(my_df)

<br><br>

## 2. Analyze the data set which already dumped into CSV file

Read the csv file `uw_cse.csv` from `./data` directory

In [229]:
my_df = pd.read_csv("./data/uw_cse.csv")
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 9 columns):
p_id              425 non-null int64
professor         425 non-null int64
student           425 non-null int64
hasPosition       425 non-null object
inPhase           425 non-null object
yearsInProgram    425 non-null object
p_id_dummy        113 non-null float64
course_id         189 non-null float64
courseLevel       189 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 30.0+ KB


Changing type of few columns

In [230]:
my_df = (my_df.astype({"p_id": np.uint16,
                      "professor": np.uint8,
                      "student": np.uint8,
                      "p_id_dummy": pd.Int32Dtype(),
                      "course_id": pd.Int32Dtype()})
        )
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 9 columns):
p_id              425 non-null uint16
professor         425 non-null uint8
student           425 non-null uint8
hasPosition       425 non-null object
inPhase           425 non-null object
yearsInProgram    425 non-null object
p_id_dummy        113 non-null Int32
course_id         189 non-null Int32
courseLevel       189 non-null object
dtypes: Int32(2), object(4), uint16(1), uint8(2)
memory usage: 19.2+ KB


Displaying the first twenty rows from `my_df`

In [231]:
my_df.head(12)

,p_id,professor,student,hasPosition,inPhase,yearsInProgram,p_id_dummy,course_id,courseLevel
0,3,0,1,0,0,0,NaN,NaN,NaN
1,4,0,1,0,0,0,NaN,NaN,NaN
2,5,1,0,Faculty,0,0,NaN,19,Level_500
3,5,1,0,Faculty,0,0,NaN,51,Level_400
4,5,1,0,Faculty,0,0,NaN,71,Level_500
5,6,0,1,0,Post_Quals,Year_2,29,NaN,NaN
6,6,0,1,0,Post_Quals,Year_2,165,NaN,NaN
7,7,1,0,Faculty_adj,0,0,NaN,NaN,NaN
8,9,0,1,0,Post_Generals,Year_5,335,124,Level_300
9,13,0,1,0,Post_Generals,Year_7,240,NaN,NaN


<br>

### Finding some insight from dataset `my_df`

### a. Professors who acted as the student advisor

In [232]:
_1 = my_df.loc[my_df["professor"]==1]
prof_id = sorted(_1["p_id"].unique())

_2 = my_df.loc[my_df["p_id_dummy"].notna()]
prof_adv_id = sorted(_2["p_id_dummy"].unique())

adv_df = (my_df.pivot_table(values="student", 
                            index=["p_id_dummy","inPhase","yearsInProgram"], 
                            columns="professor")
          .rename(columns={0: "Count"})
          .reset_index()
          .merge((my_df.set_index("p_id")
                      .loc[prof_id, :]
                      .groupby(["p_id","hasPosition"])
                      .count()
                      .reset_index()[["p_id", "hasPosition"]]),
                 how="left", 
                 left_on="p_id_dummy", 
                 right_on="p_id",
                 sort=True)
          .groupby(by=["hasPosition", "p_id_dummy", "inPhase"])
          .sum()
          .drop("p_id", axis=1)
         )
adv_df_1 = (adv_df.unstack(level=2)["Count"]
              .reset_index()
              .replace({np.nan: 0})
              .astype({"Post_Generals": np.uint8,
                       "Post_Quals": np.uint8,
                       "Pre_Quals": np.uint8})
           )


In [233]:
output_file("./bokeh_plot/advisor.html")

_x = list(zip(adv_df_1["hasPosition"].tolist(), 
              adv_df_1["p_id_dummy"].astype(str).tolist()))
_stacks = ["Pre_Quals", "Post_Quals", "Post_Generals"]
_data_ = ColumnDataSource(data=dict(
            x=_x,
            Pre_Quals=adv_df_1["Pre_Quals"].tolist(),
            Post_Quals=adv_df_1["Post_Quals"].tolist(),
            Post_Generals=adv_df_1["Post_Generals"].tolist(),
            ))

p = figure(plot_width=1200, 
           plot_height=300, 
           title="Professors who acted as the student advisor",
           x_range=FactorRange(*_x), 
           toolbar_location="above",
           tools="hover,save,reset",
           tooltips="$name: @$name student(s)",
           )

p.vbar_stack(_stacks,
             source=_data_,
             x='x',
             width=0.86,
             color=["red", "green", "blue"],
             legend=[value(item) for item in _stacks]
             )
p.x_range.range_padding = 0.01
p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = np.pi/2
p.xaxis.subgroup_label_orientation = np.pi/2
p.xaxis.axis_label = "Professor (Position -- p_id)"
p.yaxis.axis_label = "Number of students who advised"
p.outline_line_color = None
show(p)


**>>> Insight:**

In [234]:
print("Number of professors: %d persons"%(len(prof_id),))
print("Number of professors who given a task as an advisor: %d persons (%3.2f %% of %d)."%(adv_df_1.shape[0], 100*adv_df_1.shape[0]/len(prof_id), len(prof_id)))

Number of professors: 62 persons
Number of professors who given a task as an advisor: 39 persons (62.90 % of 62).


In [235]:
print("\nNumber of students advised by professors based on their position: \n")
_ = (adv_df.reset_index().groupby(["hasPosition", "p_id_dummy"]).sum().reset_index()
         .groupby("hasPosition").agg({"p_id_dummy": "count", 
                                      "Count": "sum"})
         .rename({"p_id_dummy": "Number of professors",
                  "Count": "Number of students who advised"}, axis=1))
_


Number of students advised by professors based on their position: 



,Number of professors,Number of students who advised
hasPosition,,
0,1,1
Faculty,32,86
Faculty_adj,4,7
Faculty_aff,1,2
Faculty_eme,1,1


<br><br>

### b. Courses that teached by Professors and Students

In [236]:
course_df = (my_df.loc[my_df["course_id"].notna(),:]
                .groupby(["courseLevel", "course_id"])
                .sum()
                .drop(["p_id","p_id_dummy"], axis=1)
                .reset_index()
             )


In [237]:
output_file("./bokeh_plot/courses.html")

_x = list(zip(course_df["courseLevel"].tolist(), 
              course_df["course_id"].astype(str).tolist()))
_stacks = ["professor", "student"]
_data_ = ColumnDataSource(data=dict(
            x=_x,
            professor=course_df["professor"].tolist(),
            student=course_df["student"].tolist(),
            ))

p = figure(plot_width=1500, 
           plot_height=300, 
           title="Courses that teached by professors and students",
           x_range=FactorRange(*_x), 
           toolbar_location="above",
           tools="hover,save,reset",
           tooltips="$name: @$name person(s)",
           )

p.vbar_stack(_stacks,
             source=_data_,
             x='x',
             width=0.8,
             color=["magenta", "cyan"],
             legend=[value(item) for item in _stacks]
             )
p.x_range.range_padding = 0.01
p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = np.pi/2
p.xaxis.subgroup_label_orientation = np.pi/2
p.xaxis.axis_label = "Course (Level -- course_id)"
p.yaxis.axis_label = "Number of professors/students"
p.outline_line_color = None
show(p)

In [238]:
course_pos_df = (my_df.loc[my_df["course_id"].notna(),:]
                     .groupby(["courseLevel", "course_id", "hasPosition"])
                     .agg({"professor": "sum",
                           "student": "sum"})
                     .unstack(level=2)
                     .replace({np.nan: 0})
                     .astype(np.uint16)
                     .reset_index()
                )


In [241]:
output_file("./bokeh_plot/courses_1.html")

_x = list(zip(course_pos_df["courseLevel"].tolist(), 
              course_pos_df["course_id"].astype(str).tolist()))

_stacks = ["professor_no_position", "professor_faculty", "professor_faculty_adj",
           "professor_faculty_aff", "professor_faculty_eme", "student"]
_data_ = ColumnDataSource(data=dict(
            x=_x,
            professor_no_position=course_pos_df["professor"]["0"].tolist(),
            professor_faculty=course_pos_df["professor"]["Faculty"].tolist(),
            professor_faculty_adj=course_pos_df["professor"]["Faculty_adj"].tolist(),
            professor_faculty_aff=course_pos_df["professor"]["Faculty_aff"].tolist(),
            professor_faculty_eme=course_pos_df["professor"]["Faculty_eme"].tolist(),
            student=course_pos_df["student"]["0"].tolist(),
            ))

p = figure(plot_width=1500, 
           plot_height=300, 
           title="Courses that teached by professors and students",
           x_range=FactorRange(*_x), 
           toolbar_location="above",
           tools="hover,save,reset",
           tooltips="$name: @$name person(s)",
           )

p.vbar_stack(_stacks,
             source=_data_,
             x='x',
             width=0.8,
             color=Spectral6,
             legend=[value(item) for item in _stacks]
             )
p.x_range.range_padding = 0.01
p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = np.pi/2
p.xaxis.subgroup_label_orientation = np.pi/2
p.xaxis.axis_label = "Course (Level -- course_id)"
p.yaxis.axis_label = "Number of professors/students"
p.outline_line_color = None
p.legend.orientation = "horizontal"
show(p)

**>>> Insight:**

In [240]:
print("\nNumber of professors and students who acted as lecturers per course level:\n")
(course_df.groupby("courseLevel")
     .agg({"course_id": "count",
           "professor": "sum",
           "student": "sum"})
     .rename(columns={"course_id": "Number of courses",
                      "professor": "Number of professors",
                      "student": "Number of students"})
)


Number of professors and students who acted as lecturers per course level:



,Number of courses,Number of professors,Number of students
courseLevel,,,
Level_300,7,27,3
Level_400,39,67,5
Level_500,67,86,1


There is a little student who being involved in a course by any professor on each course level.

<br><br>

### c. Professor responsibilities as the student advisors and a lecturers

In [269]:
_adv_df_ = (adv_df.reset_index()
                 .groupby(["hasPosition", "p_id_dummy"])
                 .sum().reset_index()
                 .rename(columns={"Count": "Number of student who advised",
                                  "p_id_dummy": "p_id"})
            )

In [268]:
_c_df_ = (my_df.loc[(my_df["course_id"].notna()) & (my_df["professor"]==1),:]
             .groupby(["hasPosition", "p_id", "courseLevel"])
             .agg({"course_id": "count"})
             .unstack(level=2)["course_id"]
             .replace({np.nan: 0})
             .astype(np.uint16)
             .reset_index()
          )

In [272]:
_all_ = (_c_df_.merge(_adv_df_, 
                      how="outer", 
                      left_on=["p_id", "hasPosition"], 
                      right_on=["p_id", "hasPosition"])
             .replace({np.nan: 0})
             .set_index(["hasPosition", "p_id"])
             .astype(np.uint16)
             .reset_index()
             .sort_values(["hasPosition", "p_id"])
        )
_all_.head()

,hasPosition,p_id,Level_300,Level_400,Level_500,Number of student who advised
0,0,57,2,0,0,1
1,0,64,0,1,0,0
2,0,166,0,1,0,0
3,0,181,2,0,2,0
4,0,231,1,0,0,0


In [283]:
_x = list(zip(_all_["hasPosition"].tolist(), 
              _all_["p_id"].astype(str).tolist()))

_stacks = ["Level_300", "Level_400", "Level_500"]
_data_ = ColumnDataSource(data=dict(
            x=_x,
            Level_300=_all_["Level_300"].tolist(),
            Level_400=_all_["Level_400"].tolist(),
            Level_500=_all_["Level_500"].tolist(),
            ))

p = figure(plot_width=1500, 
           plot_height=300, 
           title="Advisor and Lecturer",
           x_range=FactorRange(*_x), 
           toolbar_location="above",
           tools="hover,save,reset",
           tooltips="$name: @$name courses(s)",
           )

p.vbar_stack(_stacks,
             source=_data_,
             x='x',
             width=0.8,
             color=["red", "green", "blue"],
             legend=[value(item) for item in _stacks]
             )
# p.vbar()

p.x_range.range_padding = 0.01
p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = np.pi/2
p.xaxis.subgroup_label_orientation = np.pi/2
p.xaxis.axis_label = "Professor (Position -- p_id)"
p.yaxis.axis_label = "Number of courses"
p.outline_line_color = None
show(p)

In [282]:
_data_ = ColumnDataSource(data=dict(
            x=_x, 
            student=_all_["Number of student who advised"].tolist()
            ))
p = figure(plot_width=1500, 
           plot_height=300, 
           title="Advisor and Lecturer",
           x_range=FactorRange(*_x), 
           toolbar_location="above",
           tools="hover,save,reset",
           tooltips="$name: @$name courses(s)",
           )
p.vbar(x='x',
       top="student",
       width=0.8,
       source=_data_)

p.x_range.range_padding = 0.01
p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = np.pi/2
p.xaxis.subgroup_label_orientation = np.pi/2
p.xaxis.axis_label = "Professor (Position -- p_id)"
p.yaxis.axis_label = "Number of students who advised"
p.outline_line_color = None
show(p)